In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:

url = "https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports/current/countrybycommodityexports.zip"

In [3]:
r = requests.get(url) 

brexit_Jan21 = zf.ZipFile(io.BytesIO(r.content))

brexit_Jan21.extractall(".\\data")

filename = brexit_Jan21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Export")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,1997JAN,1997FEB,1997MAR,1997APR,1997MAY,1997JUN,1997JUL,...,2020APR,2020MAY,2020JUN,2020JUL,2020AUG,2020SEP,2020OCT,2020NOV,2020DEC,2021JAN
0,0 Food & live animals,AD Andorra,EX Exports,0.01,0.04,0.07,0.01,0.04,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00
1,0 Food & live animals,AE United Arab Emirates,EX Exports,4.18,3.93,4.26,4.61,3.77,4.90,3.80,...,18.86,14.99,13.71,15.44,16.95,19.71,21.88,19.08,22.68,15.35
2,0 Food & live animals,AF Afghanistan,EX Exports,2.30,0.00,0.00,0.00,0.01,0.00,0.00,...,0.00,0.14,0.16,0.08,0.03,0.11,0.12,0.06,0.45,0.13
3,0 Food & live animals,AG Antigua & Barbuda,EX Exports,0.17,0.13,0.22,0.27,0.19,0.23,0.32,...,0.02,0.21,0.06,0.14,0.18,0.19,0.25,0.23,0.30,0.23
4,0 Food & live animals,AI Anguilla,EX Exports,0.00,0.00,0.01,0.03,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "exports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.exports = df.exports*1000000

In [7]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"exports": "sum"})

In [8]:
topcntry = test.sort_values(by = "exports", ascending = False).iloc[0:23].index.to_list()

In [9]:
df = df[df.COUNTRY.isin(topcntry)]

In [10]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)

In [11]:
topcntry

['Whole world',
 'Total EU(28)',
 'Extra EU 28 (Rest of World)',
 'United States inc Puerto Rico',
 'Germany',
 'France',
 'Netherlands',
 'Ireland',
 'Belgium',
 'Spain',
 'Italy',
 'China',
 'Switzerland',
 'Sweden',
 'Hong Kong',
 'Japan',
 'United Arab Emirates',
 'Canada',
 'India',
 'Singapore',
 'Australia',
 'Turkey',
 'Poland']

In [12]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [13]:
df.head()

,,,exports
COUNTRY,COMMODITY,time,
United Arab Emirates,0 Food & live animals,1997-01-01,4180000.0
Australia,0 Food & live animals,1997-01-01,4950000.0
Total EU(28),0 Food & live animals,1997-01-01,385020000.0
Belgium,0 Food & live animals,1997-01-01,22550000.0
Canada,0 Food & live animals,1997-01-01,8180000.0


In [14]:
out_file = ".\\data"+ "\\UK-exports.parquet"

pq.write_table(pa.Table.from_pandas(df.sort_index()), out_file)

In [15]:
file = ".\\data"+ "\\UK-exports.parquet"

foo = pq.read_table(file).to_pandas()

In [16]:
foo.head()

exports
COUNTRY   COMMODITY             time                 
Australia 0 Food & live animals 1997-01-01  4950000.0
                                1997-02-01  4870000.0
                                1997-03-01  3800000.0
                                1997-04-01  4630000.0
                                1997-05-01  4230000.0